In [1]:
from utils.models_onehot import GMF, MLP, NeuFM

In [2]:
from utils.dataset import RatingsDatasetOneHoteHot
from torch.utils.data import DataLoader

In [3]:
users = 3974
movies = 3564

dataloader = DataLoader(
    RatingsDataset(
        "/home/nubol23/Desktop/Codes/USP/SCC5966/kaggle/data/train_data.csv",
        "user_id",
        "movie_id",
        "rating",
    ), 
    batch_size=256,
    num_workers=3
)

In [4]:
import torch.nn as nn
from torch.optim import SGD
from tqdm.notebook import tqdm, trange

In [9]:
gmf = GMF(users, movies, 50).cuda()

In [10]:
criterion = nn.MSELoss().cuda()
optimizer = SGD(gmf.parameters(), lr=0.01)

for epoch in range(1):
    n_batches = len(dataloader)
    avg_loss = 0
    for i_batch, (vus, vis, rs) in tqdm(enumerate(dataloader), total=n_batches):
        vus = vus.cuda()
        vis = vis.cuda()
        rs = rs.cuda()
        
        optimizer.zero_grad()
        y_hat = gmf(vus, vis)
        
        loss = criterion(y_hat, rs)
        loss.backward()
        
        optimizer.step()
        
        avg_loss += float(loss.detach().cpu())
    print(avg_loss/n_batches)

  0%|          | 0/2093 [00:00<?, ?it/s]

14.221903687279816


In [11]:
for vus, vis, rs in dataloader:
    vus = vus.cuda()
    vis = vis.cuda()
    
    y_hat = gmf(vus, vis)
    print(y_hat.ravel())
    print(rs.ravel())
    
    break

tensor([-4.8413e-03,  9.8179e-03,  7.6315e-03, -3.7077e-04,  7.0648e-03,
         7.9945e-03, -1.7278e-03,  1.1538e-03,  4.5402e-03, -3.0193e-04,
        -1.3401e-03,  5.6100e-03,  2.5313e-03, -1.7431e-03, -1.1212e-02,
        -6.9852e-03, -4.0421e-03,  9.6643e-03, -8.6843e-03, -6.3857e-03,
        -8.4091e-03, -6.3270e-03, -3.9746e-04, -4.6516e-03, -1.8829e-03,
         1.1957e-02, -3.5038e-03, -3.0149e-03, -9.7645e-03,  2.0909e-03,
        -1.2883e-03, -7.5644e-03, -5.0838e-04, -4.6313e-03,  2.9897e-03,
         8.4702e-03, -5.6593e-03, -4.7498e-04,  8.4371e-06, -2.4144e-03,
         6.9013e-05, -3.8061e-03, -3.8925e-03,  2.3555e-03,  1.0866e-02,
         2.5534e-03,  3.2060e-03, -5.6483e-03, -3.9527e-03, -3.8850e-03,
         6.7560e-03, -7.2027e-03, -1.3746e-03,  5.2484e-03,  4.3327e-03,
         5.1155e-03, -6.4491e-03, -1.3897e-02,  7.8402e-03, -6.0865e-03,
         6.4076e-03,  1.8263e-02, -4.6735e-03,  5.7296e-03,  6.5046e-03,
         1.0981e-02,  2.4326e-04, -1.4758e-03, -1.2

In [ ]:
# mlp = MLP(users, movies, 5, [20, 10])

# neufm = NeuFM(gmf, mlp, 0.5)